<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/VGG16_CE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 527 kB/s 
     |████████████████████████████████| 419 kB 14.7 MB/s 
Tue Sep 20 09:28:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |  

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(43)

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Data pre-processing**

In [4]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=[32,32]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = datasets.CIFAR100(root='data/', download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
num_classes = 100
num_epochs = 100

In [6]:
class VGGForCiFar100(nn.Module):
  def __init__(self, hid_dim=5000, dropout=0.5, n_classes=100, use_fc=True, freeze=False):
    super().__init__()
    self.vgg = models.vgg16(pretrained=True)
    if not use_fc:
      self.vgg.classifier = nn.Linear(512 * 7 * 7, n_classes)
    else:
      self.vgg.classifier = nn.Sequential(
              nn.Linear(512 * 7 * 7, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, n_classes),
          )
      
    if freeze:
      for param in self.vgg.features.parameters():
        param.requires_grad = False
      for param in self.vgg.avg_pool.parameters():
        param.requires_grad = False
      for param in self.vgg.flatten.parameters():
        param.requires_grad = False
    

  def forward(self, x):
    return F.log_softmax(self.vgg(x), dim=-1)

**VGG16 Architecture**

In [7]:
model = VGGForCiFar100(n_classes=num_classes).to(device)
criterion = nn.NLLLoss().to(device)
accuracy = Accuracy(num_classes=num_classes).to(device)
optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

**Training and Validation**

In [8]:
total_train_step = len(train_loader)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        # Move tensors to the configured device
        images = images.to(device)
        target = target.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, target)

        train_loss += loss
        train_acc += accuracy(outputs, target)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):
            images = images.to(device)
            target = target.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, target)
            val_acc += accuracy(outputs, target)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

/usr/local/lib/python3.7/dist-packages/pytorch_ranger/ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1174.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch [1/100] - Loss: 3.0451, Accuracy: 0.2620
Accuracy of the network on the 5000 validation images: 0.4691, loss: 1.9273
Epoch [2/100] - Loss: 1.9695, Accuracy: 0.4654
Accuracy of the network on the 5000 validation images: 0.5269, loss: 1.6579
Epoch [3/100] - Loss: 1.6835, Accuracy: 0.5352
Accuracy of the network on the 5000 validation images: 0.5676, loss: 1.5460
Epoch [4/100] - Loss: 1.4963, Accuracy: 0.5845
Accuracy of the network on the 5000 validation images: 0.5823, loss: 1.4930
Epoch [5/100] - Loss: 1.3662, Accuracy: 0.6145
Accuracy of the network on the 5000 validation images: 0.6020, loss: 1.4458
Epoch [6/100] - Loss: 1.2600, Accuracy: 0.6432
Accuracy of the network on the 5000 validation images: 0.6072, loss: 1.4113
Epoch [7/100] - Loss: 1.1710, Accuracy: 0.6662
Accuracy of the network on the 5000 validation images: 0.6269, loss: 1.3690
Epoch [8/100] - Loss: 1.0880, Accuracy: 0.6898
Accuracy of the network on the 5000 validation images: 0.6339, loss: 1.3491
Epoch [9/100] - 

**Testing**

In [9]:
# model.save(BEST_MODEL, "./vgg16_cifar100_best.pt")
# model.load_state_dict(model.load("./vgg16_cifar100_best.pt"))
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):

        # Move tensors to the configured device
        images = images.to(device)
        target = target.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,target)
        test_acc += accuracy(outputs, target)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.6653, loss: 1.8612
